## Functions and Functional Programming

### Concepts
* free functions
  + fucntions defined at module scope
* methods
  + fucntions defined within a class definition
* argument types
  + the choice between positional and keyword arguments is made at the call site, not at the definition
  + a particular argument may be passed as a positional argument in one call, but as a keyword argument in another call
  * positional arguments
    + matched with formal arguments by position in order
  * keyword arguments
    + matched with formal arguments by name, can be provided in any order, if provided following all positional arguments 
* default arguments
  + arguments may have a default value
  + the righ hand side of the default value is only evaluated once at the time enclosing def statement is executed, which usually is when the module is first imported
    + be careful when using mutable default values, which can inadvertently retain modifications between calls
* functions are objects and can be passed around just like any other objects 
* function objects are callable objects
* def keyword is responsible for binding a function object, which contains a function definition, to a function name
* see the following cell for code example
  + function name: resolve refers to a function object
  + use postfix parentheses (function call operator) to invoke function

In [1]:
import socket
def resolve(host):
    return socket.gethostbyname(host)

In [2]:
resolve

<function __main__.resolve(host)>

In [3]:
resolve('sixty-north.com')

'93.93.131.30'

#### callable objects
\_\_call\_\_()
* allows instances of classes to be callable objects
* is invoked on objects when thare are called like functions
* in the following cell, Resolver is a class with its \_\_call\_\_() method implemented
* a Resolver object can be called, using the argument defined in its \_\_call\_\_(), which is host (self does not count)
* in another word, resolver(host) == resolver.\_\_call\_\_(host)
* since callable instances are just normal class instances, their classes can define any other methods you want, such as the clear() and has_host()
* as demonstrated in the code, we use \_\_call\_\_ when we need to maintain state information between calls, here is the queried ip hosts and addresses

In [5]:
import socket

class Resolver:
    def __init__(self):
        self._cache = {}

    def __call__(self, host):
        if host not in self._cache:
            self._cache[host] = socket.gethostbyname(host)
        return self._cache[host]

    def clear(self):
        self._cache.clear()

    def has_host(self, host):
        return host in self._cache

In [6]:
resolver = Resolver()
resolver("sixty-north.com")

'93.93.131.30'

In [14]:
# resolver(host) == resolver.__call(host)
resolver.__call__("sixty-north.com")

'93.93.131.30'

In [8]:
resolver._cache

{'sixty-north.com': '93.93.131.30'}

In [12]:
resolver('pluralsight.com')

'35.162.133.205'

In [15]:
# resolver._cache stores all the queried host ips in the dictionary
resolver._cache

{'sixty-north.com': '93.93.131.30', 'pluralsight.com': '35.162.133.205'}

#### Show the time speed up by using cache

In [16]:
from timeit import timeit
timeit(setup="from __main__ import resolve", stmt="resolve('google.com')", number=1)

0.031369100004667416

In [18]:
# the second time it is called, it is much faster since the results are retrieved from cache in memory
timeit(setup="from __main__ import resolve", stmt="resolve('google.com')", number=1)

0.0004041000211145729

#### clear the cache by clear() method

In [19]:
print("before clearing cache")
print(resolver.has_host("pluralsight.com"))
resolver.clear()
print("after clearing cache")
print(resolver.has_host("pluralsight.com"))

before clearing cache
True
after clearing cache
False


### Classes are callable
* not only the class instances, but also the class objects themselves are callable
* class objects and instances of classes are very different things
* in python the 'class' keyword binds a class object to a named referencing that class
* when we consruct a new instance, we call the class object of the corresponding class
  + Resolver() where Resolver is the class object, and we call it by postfix parenthesis, and this call is forwarded to the class \_\_init\_\_() method, if defined
* classes are object factories and they produce new instances when they are invoked   

#### Returning class objects
* the following code returns class object.
  + both tuple and list are built-in class objects
* we then call these class objects and construct the corresponding class instances
  + when cls== tuple, it creates a tuple object, otherwise, a list object

In [21]:
def sequence_class(immutable):
    if immutable:
        cls = tuple
    else:
        cls = list
    return cls    

In [22]:
seq = sequence_class(immutable=True)
t = seq("Timbuktu")
t

('T', 'i', 'm', 'b', 'u', 'k', 't', 'u')

#### Conditional experssions
* evaluate to one of two expressions depending on a boolean
* result = true_value if condition else false_value
* we can use this to modify our sequence_class code

In [23]:
def sequence_class(immutable):
    return tuple if immutable else list

seq = sequence_class(immutable=True)
t = seq("Timbuktu")
t

('T', 'i', 'm', 'b', 'u', 'k', 't', 'u')

#### lambda allows you to create anonymous callable objects
* for example, sorted(iterable, key)
  + key is a callable such as a lambda
  + iterable such as a list
* lambda defines a function, if you assign a lambda to a variable, that variable will be a callable function  

In [26]:
scientists = ['Marie Curie', 'Albert Einstein', 'Rosalind Franklin',        
             'Niels Bohr', 'Dian Fossey', 'Isaac Newton',                  
              'Grace Hopper', 'Charles Darwin', 'Lise Meitner']             
sorted(scientists, key=lambda x: x.split()[-1])                                                                      

['Niels Bohr',
 'Marie Curie',
 'Charles Darwin',
 'Albert Einstein',
 'Dian Fossey',
 'Rosalind Franklin',
 'Grace Hopper',
 'Lise Meitner',
 'Isaac Newton']

In [27]:
last_name = lambda x: x.split()[-1]
last_name

<function __main__.<lambda>(x)>

#### More about lambda
* lambda uses expression which evaluates to a function, a regular function uses statement
* lambda is anonymous
* argument list terminated by a colon and separated by commas
* zero or more arguments supported (zero arguments -> lambda:
* boday is a single expression
* no return statement is allowed, return value is given by the body expression 

#### Detecting Callable objects
* using callable() to detect if an object is callable
* function is callable
* lambda is callable
* method is callable
* class object is callable since they can be called to construct objects
* instance variable of a class implementing \_\_call\_\_() method is callable

### Extended Argument Syntax
* applies to all types of callables
#### positional arguments (as \*args)
  + the type of \*args is a tuple
  + when you need to accept a variable number of arguments with a positive lower bound you should consider to use regular positional arguments for the required parameters and \*args to deal with any extra arguments
  + \*args must come after normal positional arguments
  + can have only one \*args in the argument list
  + only collects positional arguments

In [28]:
def hypervolume(*args):
    print(args)
    print(type(args))

In [29]:
hypervolume(3, 4)

(3, 4)
<class 'tuple'>


In [30]:
# version using iter 
def hypervolume(*lengths):
    i = iter(lengths)
    v = next(i)
    for length in i:
        v *= length
    return v    

In [31]:
hypervolume(3, 4, 5)

60

In [32]:
# more straightforward version using iterator
def hypervolume(length, *lengths):
    v = length
    for item in lengths:
        v *= item
    return v    

In [33]:
hypervolume(3, 4, 5)

60

#### arbitrary keyword arguments
* prefix argument with \*\* to accept arbitrary keyword arguments
* conventionally called \*\*kwargs
  

In [34]:
def tag(name, **kwargs):
    print(name)
    print(kwargs)
    print(type(kwargs))

In [35]:
tag('img', src='Monet.jpg', alt="Sunrise by Claude Monet", border=1)

img
{'src': 'Monet.jpg', 'alt': 'Sunrise by Claude Monet', 'border': 1}
<class 'dict'>


In [36]:
def tag(name, **attributes):
    result = '<' + name                                                     
    for key, value in attributes.items():
        result += ' {k}="{v}"'.format(k=key, v=str(value))                  
    result += '>'                                                           
    return result              

In [37]:
tag('img', src='Monet.jpg', alt="Sunrise by Claude Monet", border=1)

'<img src="Monet.jpg" alt="Sunrise by Claude Monet" border="1">'

#### combine positional and arbitrary keyword arguments
* \*args, if present, must before \*\*kwargs
* any arguments preceding \*args are token to be regular positional arguments
* any regular arguments after \*args must be passed as mandatory keyword arguments
* to use mandatory keyword arguments without accepting arbitrary number of positional arguments, python allows to omit the name of \*args argument
* arguments following the asterisk can only be passed to the function as keyword arguments, and you are not allowed to pass extra positional arguments to the function
* \*\*kwargs, if present, must be the last in the argument list

In [38]:
def name_tag(first_name, last_name, *, title=''):
    print(title, first_name, last_name)
    
# accept the first_name, last_name as positional 
# and use keyword arg title without accepting arbitrary number of positional argument
name_tag('Judy', 'Spudmeyer', title='Galactic Commander')     

In [39]:
# can not pass any extra positional arg if use *
name_tag('James', 'Tiberius', 'Kirk', title='Capt.') 

TypeError: name_tag() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [41]:
 # all args after *args must be keyword args, 
 # and **kwargs, if present, must be the last one in the arg list
def print_args(arg1, arg2, *args, kwarg1, kwarg2, **kwargs):
        print(arg1)                                                             
        print(arg2)                                                             
        print(args)                                                             
        print(kwarg1)                                                           
        print(kwarg2)                                                           
        print(kwargs)                                                           
                                                                            
print_args(1, 2, 3, 4, 5, kwarg1=6, kwarg2=7, kwarg3=8, kwarg4=9)     

1
2
(3, 4, 5)
6
7
{'kwarg3': 8, 'kwarg4': 9}


#### Default arguments
* when combined with default arguments, mandatory args must be specified before optional arguments at the call site

#### positional-only arguments
* positional only arguments can be defined by using a / in the arg list. Any args before / must be positional, and can not be passed as keyword arg

In [42]:
def number_length(x, /):
    return len(str(x))

In [43]:
number_length(2112)

4

In [44]:
number_length(x=2345)

TypeError: number_length() got some positional-only arguments passed as keyword arguments: 'x'

### Extended call syntax
* this complements to the extended formal argument syntax
* allows us to use iterable series, such as a tuple, to populate positional arguments and any mapping type, suh as a dictionary, that has string keys to populated keyword arguments
* in the following code, the tuple are unpaced into the positional argument list at call site
* 

In [51]:
# example of unpacking input tuple
def print_args(arg1, arg2, *args):
    print("------------------------------")
    print("demo unpacking tuple input to argument list")
    print(arg1)
    print(arg2),
    print(args)
    
# construct input tuple
t = (11, 12, 13, 14)

# pass the tuple and unpack it to argument list
print_args(*t)



# example of unpacking input directionary
def color(red, green, blue, **kwargs):
    print("------------------------")
    print("demo unpacking input dictionary to argument list")
    print("r =", red)
    print("g =", green)
    print("b =", blue)
    print(kwargs)
    
# construct input dictionary
k = {'red': 21, 'green': 68, 'blue': 120, 'alpha': 52}

# pass the dictionary and unpack it to the argument list
color(**k)

k1 = dict(red=21, green=68, blue=120, alpha=52)
color(**k1)

------------------------------
demo unpacking tuple input to argument list
11
12
(13, 14)
------------------------
demo unpacking input dictionary to argument list
r = 21
g = 68
b = 120
{'alpha': 52}
------------------------
demo unpacking input dictionary to argument list
r = 21
g = 68
b = 120
{'alpha': 52}


#### Argument Forwarding
* use both star args and start star args to forward all arguments of a function to another function
* the format is usually \*args, \*\*kwargs. Note that \*args should always preceeding \*\*kwargs

In [55]:
def trace(f, *args, **kwargs):
    print("args =", args)
    print("kwargs =", kwargs)
    result = f(*args, **kwargs)
    print("result =", result)
    return result

# call trace and pass "ff" and base=16 to int function
trace(int, "ff", base=16)

args = ('ff',)
kwargs = {'base': 16}
result = 255


255

### Closure
* python support for local functions: functions to find within other functions
* enclosing namespace (come into play when loacal functions are used)
* how local functions are returned from functions just like other functions
* how closures are used for managing object lifetimes for local functions
* how to use nonlocal keyword to pull names from a enclosing namespace into local functions,similar to how the global keyword works  

#### How functions are defined
* use 'def' to define new function
* 'def' binds a function object (the function body) to a name (function name)
* 'def' is executed at run time, meaning that functions are defined at runtime 
* functions can be defined at module-level or within a class as class methods
* you can also define functions inside other functions
  + these functions are called local functions since they are defined local to a specific function's scope
  + an example is shown in the following cell: function last_letters is defined and used within function sort_by_last_letter

In [56]:
def sort_by_last_letter(strings):
    def last_letter(s):
        return s[-1]
    return sorted(strings, key=last_letter)

sort_by_last_letter(['hello', 'from', 'a', 'local', 'function'])

['a', 'local', 'from', 'function', 'hello']

#### execution of local functions
* local functions are defined at runtime when def is executed
* in the above example, each time when sort_by_last_letter is called, a new copy of last_letter function is created locally for each enclosing invocation
* each time when sort_by_last_letter is called, a new local function instance is created and bound to last_letter

#### scopes of local functions for name resolution
* local function follows the same LEGB rule for name lookup from local, enclosing, global and built-in scopes
* local functions find names 
  + start with names in the local function itself
  + then checks enclosing scope(s), which is the containing function both the local names and parameters of the containing function
  + finally module-leve (which is the global) and built-in names are checked
* local functions are not members of their enclosing functions. They are just local name bindings in the function body 
  + in the following example, you can not access inner function from outer function by outer.inner(), since inner is not the member of outer. It is just a variable defined in outer function's body

In [58]:
# module-level (global)
g = 'global'

# enclosing-level param
def outer(p='param'):
    l = 'local'  # local variable
    def inner():
        print(g, p, l)
    inner()
    
outer()    


global param local


#### When to use local function
* defining one-off fucntions close to their use
* assist code organization and readability similar to lambdas, but more general
  + they can contain multiple expressions
  + may contains statements such as import
* 

#### First-class functions
* functions can be passed to and returned from functions
* more generally, they can be treated like any other data
* this will be a powerful concept when combined with closures
* local functions can be returned from the enclosing functions
* in the following code example, local function, local_func is returned from enclosing()
* when we execute enclosing(), we assigned the returned local function to a variable, lf, which is a function
* we then execute the lf function by lf()

In [59]:
def enclosing():
    def local_func():
        print('local func')
    return local_func

lf = enclosing()
lf()

local func


#### Closures and nested scopes
* how does a local function find enclosing variables when enclsing scope is gone once the local function is returned?
  + by closure.
  + a closure essentially records the objects from the enclosing scope that the local function needs
  + it then keeps recorded objects alive for use after the enclosing scope is gone
* python implements closure with the \_\_closure\_\_ attribute
  + if a function closes over any objects, then that function has a \_\_closure\_\_ attribute, which maintain the necessary references to those objects, and prevent those objects from garbage collections, as shown in the following cell

In [60]:
def enclosing():
    x = 'closed over'
    def local_func():
        print(x)
    return local_func

lf = enclosing()
lf()
lf.__closure__

closed over


(<cell at 0x000002E3DDAB7100: str object at 0x000002E3DF5301B0>,)

#### Function factories
* functions that return other functions, with returned functions specialized based on the arguments to the factory
* returned functions use both their own arguments as well as arguments to the factory
  + combination of runtime function definition and closures makes this possible
* the following code example shows a function factory that generate functions to raises numbers to a particular power

In [61]:
def raise_to(exp):
    def raise_to_exp(x):
        return pow(x, exp)
    return raise_to_exp

square = raise_to(2)
square.__closure__

(<cell at 0x000002E3DDAB7220: int object at 0x000002E3D9020110>,)

In [63]:
print(square(5))
print(square(9))
print(square(11))

25
81
121


#### Binding names in enclosing scopes
* if we have a module or enclosing level variable, and when you assign a new value to it in our local function:
  + local function will create a new local variable with the same name, and assign the value to it without changing the module or enclosing variable value
  + if we want local function to change a module level variable, we can bind the global variabel in to local()
* similarly, if we want to bind an enclosing scope variable to local function, we use nonlocal
  + nonlocal inserts a name binding from an enclosing scope into the local namespace
  + python searches enclosing scopes from innermost to outermost and use the first match found
  + if there is no matching of the enclosing scope variable, python will raise a syntax error

In [67]:
# local function will create a new variable(message) rathe than modifying global or enclosing level message variables
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        message = 'local'
        
    print("enclosing message:", message)
    local()
    print("encolsing message", message)
    
print("global message:", message)
enclosing()   
print("global message:", message)

global message: global
enclosing message: enclosing
encolsing message enclosing
global message: global


In [69]:
# to modify the global variable, we need to declare message variable as global and then modify it
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        global message
        message = 'local'
        
    print("enclosing message:", message)
    local()
    print("encolsing message", message)
    
print("global message:", message)
enclosing()   
print("global message:", message)

global message: global
enclosing message: enclosing
encolsing message enclosing
global message: local


In [72]:
# to modify the enclosing variable, we need to declare message variable as nonlocal and then modify it
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        nonlocal message
        message = 'local'
        
    print("enclosing message:", message)
    local()
    print("encolsing message", message)
    
print("global message:", message)
enclosing()   
print("global message:", message)

global message: global
enclosing message: enclosing
encolsing message local
global message: global


#### Reading from global and enclosing scope
* for local functions to read from a variable name, it follows LEGB order

In [71]:
# when reading variables, local function follows LEGB
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        print("local function", message)        
    print("enclosing message:", message)
    local()
    print("encolsing message", message)
    
print("global message:", message)
enclosing()   
print("global message:", message)

global message: global
enclosing message: enclosing
local function enclosing
encolsing message enclosing
global message: global


#### an example of using enclosing variable by nonlocal
* last_called is created as a local variable of make_timer() function
* when make_timer() is called, it creates a local function elapsed(), which connects to last_called
* by calling make_timer(), a new copy of elapsed() function is returned and stored in variable t
* each time when t() is executed, it will calculate differnce between its new local variable, now and last_called, and update the value of last_called
  + basically, this function returns the time elapsed between the current time and last time funtion elapsed() is called. The historical call time was stored in last_called enclosing variable
 * when you call make_timer() again, a new copy of elapsed() function will be created as an independent timer 

In [85]:
import time
import sys
def make_timer():                                                           
    last_called = None # Never                                              
    def elapsed():                                                          
        nonlocal last_called                                                
        now = time.time()                                                   
        if last_called is None:                                             
            last_called = now                                               
            return None                                                     
        result = now - last_called                                          
        last_called = now                                                   
        return result                                                       
    return elapsed                                                          
                                                                            
t = make_timer()               

In [88]:
print(t())
time.sleep(1)
print(t())
time.sleep(2)
print(t())
time.sleep(3)
print(t())

28.10009479522705
1.0095770359039307
2.0030558109283447
3.005105972290039


### function decorators
* add behaviors to existing functions without modifying the functions
* apply decorators to functions
* use any callable as a decorator
* apply mutiple decorators
* semantics of such application
* standard library support for decorators
* parameterized decorators

#### decorators
* modify or enhance an existing function in a non-intrusive and maitainable way
* implemented as a callable that accepts a callable and returns a callable
* for a typical decorated function as defined in the following:
```python
@my_decorator
def my_function():
    pass
```
  + my_function is compiled to generate a function object
  + the function object is passed to decorator (my_decorator), which is a function that accepts my_function as input, and returns a function
  + this decorator is then called, and python will bind the returned function to my_function
  + when my_function is called, the function now bound to my_function will execute
  + the result is that decorators allow you to modify existing functions without changing their definitions
  + callers don't need to change when decorators are applied, because the decorator mechanism ensures that the same name is applied for both decorated and un-decorated function
  * the following is a simple code example for decorators to convert the results generated by input functions to ascii

In [93]:
def northern_city():
    return 'Troms╟'

print(northern_city())

@escape_unicode
def northern_city():
    return 'Troms╟'

print(northern_city())

Troms╟
'Troms\u255f'


#### Using classes as decorators
* classes as decorators
  + functions decorated with a class are replaced by an instance of the class
  + these instances must themselves be callable, meaning that the class implements \_\_call\_\_() method
  + in the following code example:
    + class CallCount decorates function hello(), so 'hello' now refers to a CallCount instance, and each time hello() function is called, it will execute \_\_call\_\_() instance function, this function first increments the count, and then execute original hello() function, and returns the results
    + The key point here is that when decorating hello() by CallCount, the compiled hello() function object is passed to CallCount constructor, which is the \_\_init\_\_(self, f), and therefore, returns a CallCount instance, which is then assinged/bound to 'hello'
    + when hello(name) is called, the instance method \_\_call\_\_(name) is invoked, which increments count, and returns the execution results of original function of hello(name)
    + now, when we run hello.count, we get the final value of count

In [97]:
class CallCount:
    def __init__(self, f):
        self.f = f
        self.count = 0
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.f(*args, **kwargs)

@CallCount
def hello(name):
    print('Hello, {}'.format(name))

# execute decorated hello() function
hello('Fred')
hello('Wilma')
hello('Betty')
hello('Barney')
print(hello.count)

Hello, Fred
Hello, Wilma
Hello, Betty
Hello, Barney
4


#### using instances as decorators
* python calls an instances \_\_call\_\_() when it's used as a decorator
* \_\_call\_\_()'s return value is used as the new function
* create groups of callables that you can dynamically control as a group
* in the following code example, 
  + when rotate_list is decorated by tracer instance, its \_\_call\_\_() method is invoked, with rotate_list() function passed to it. 
  + The returned wrap function is then assigned to rotate_list()
  + when rotate_list is called, wrap is executed by printing the 'Calling f' statement and the returns the results of executing the original rotate_list() function


In [102]:
class Trace:    
    def __init__(self):                                                     
        self.enabled = True                                                 
    def __call__(self, f):                                                  
        def wrap(*args, **kwargs):                                          
            if self.enabled:                                                
                print('Calling {}'.format(f))                               
            return f(*args, **kwargs)                                       
        return wrap                                                         
                                                                            
tracer = Trace()

@tracer                                                                     
def rotate_list(l):                                                         
    return l[1:] + [l[0]]    

l = [1, 2, 3]
l = rotate_list(l)
print(l)

l = rotate_list(l)
print(l)

l = rotate_list(l)
print(l)

# disable the enabled label to not print tracing information
tracer.enabled = False
l = rotate_list(l)
print(l)

l = rotate_list(l)
print(l)


Calling <function rotate_list at 0x000002E3DF57F2E0>
[2, 3, 1]
Calling <function rotate_list at 0x000002E3DF57F2E0>
[3, 1, 2]
Calling <function rotate_list at 0x000002E3DF57F2E0>
[1, 2, 3]
[2, 3, 1]
[3, 1, 2]


#### Multiple decorators
* multiple decorators are processed in the reversed order
* in the following example
  + some_function is passed to decorator3, with its returned function passed to decorator2, with its returned function passed to decorator1
  + the returned function from decorator1 is assigned to some_function

In [ ]:
# Using multiple decorators
@decorator1
@decorator2
@decorator3
def some_function():
    pass

#### Decorate class methods
* we can decorate class methods as we did on functions
* as shown in the following code, trace instance works well with class methods

In [104]:
class IslandMaker:
    def __init__(self, suffix):
        self.suffix = suffix
    @tracer
    def make_island(self, name):
        return name + self.suffix
    
im = IslandMaker(' Island')
tracer.enabled = True
im.make_island('Python')

Calling <function IslandMaker.make_island at 0x000002E3DF5E3AC0>


'Python Island'

#### Losing Metadata
* with the use of decorators, we lose important metadat from the original callables
* this is because, now the original function name is bounded to the results of the returned function of the deocraor
* how to keep the name and doc string of the original function?
  + we can copy __name__ and __doc__ from wrapped function to our wrapper function
  + use functools.wraps()
    + is a decorator by itself
    + apply to wrapper function and takes the decorated function as the input
    + replace decorator metadata with that of the decorated callable

In [106]:
def hello():
    "Print a well-known message"
    print('Hello, world!')
    
print(hello.__name__)
print(hello.__doc__)
help(hello)

hello
Print a well-known message
Help on function hello in module __main__:

hello()
    Print a well-known message



In [108]:
# with a decorator, the original metadata about this function is lost
def noop(f):
    def noop_wrapper():
        return f()
    return noop_wrapper

@noop
def hello():
    "Print a well-known message"
    print('Hello, world!')
    
print(hello.__name__)
print(hello.__doc__)
help(hello)    
    

noop_wrapper
None
Help on function noop_wrapper in module __main__:

noop_wrapper()



In [109]:
# copy the __name__ and __doc__ from wrapped to wrap function
# to maintain the meta  data of original function
def noop(f):
    def noop_wrapper():
        return f()
    noop_wrapper.__name__ = f.__name__
    noop_wrapper.__doc__ = f.__doc__
    return noop_wrapper

@noop
def hello():
    "Print a well-known message"
    print('Hello, world!')
    
print(hello.__name__)
print(hello.__doc__)
help(hello)    
    

hello
Print a well-known message
Help on function hello in module __main__:

hello()
    Print a well-known message



In [111]:
# using functools.wraps to maintain the meta data of decorated functions
import functools

def noop(f):
    @functools.wraps(f)
    def noop_wrapper():
        return f()
    return noop_wrapper

@noop
def hello():
    "Print a well-known message"
    print('Hello, world!')
    
print(hello.__name__)
print(hello.__doc__)
help(hello)    

hello
Print a well-known message
Help on function hello in module __main__:

hello()
    Print a well-known message



#### Parameterized Decorators
* the following decorator accepts the index as the argument, and then checks the argument at that index to make sure it is non-negative value
* how does this work?
  + check_non_negative itself is not a decorator, since a decorator needs to accept a callable and returns a callable
  + the key point here is that when we decorate the create_list, we call the check_non_negative function, and it is the function it returns (validator()), is the decorator
  + decorator function (validator()) accepts the decorated function as its argument 
  + python calls decorator function (validator(f)), and associate the decorated function name (create_list) to the returned function object, which is wrap()
  + when you call create_list, wrap() function will be called, with the access to both f(original create_list function object), and index, as well as the arguments when calling the function

In [119]:
import functools
def check_non_negative(index): 
    def validator(f):
        @functools.wraps(f)
        def wrap(*args):                                                    
            if args[index] < 0:                                             
                raise ValueError(                                           
                    'Argument {} must be non-negative.'.format(index))      
            return f(*args)                                                 
        return wrap                                                         
    return validator                                                        
                                                                            
@check_non_negative(1)                                                      
def create_list(value, size): 
    "create a list of given value and size"
    return [value] * size                                                   
                                                                            
create_list('a', 3)                                                         
['a', 'a', 'a']                                                                 
   

['a', 'a', 'a']

In [120]:
help(create_list)

Help on function create_list in module __main__:

create_list(value, size)
    create a list of given value and size



In [113]:
create_list(123, -6)          

ValueError: Argument 1 must be non-negative.

#### Functional-stype Tools
##### map()
  + calls a function for the elements in a sequence, producing a new sequence with the returned values
  + it maps a function over a sequence
  + map() will not call its function or access its iterables until they are needed for output
  + a map object is iteself iterable, iterate over it to proeduce output
  + map can use with as many input sequences as your mapped function needs.
    + you need to input as many input sequences as the number of arguments in your mapped function
    + map will take one element from each of the sequence, and pass them to the mapped function to produce value
    + if sequences have different length, map will terminate as the shortest sequence is terminated

##### Map function examples

In [130]:
# example of map()
# find the ord of each char in the input string(which is a tuple/list)
# ane return them as a sequence/generator in the same order
for o in map(ord, 'The quick brown fox'):
    print(o)

84
104
101
32
113
117
105
99
107
32
98
114
111
119
110
32
102
111
120


in the following code, we use map to call Trace() constructor, which returns a Trace instance. 
This instance is than called by postfix parenthesis '()', with the argument of ord function

In [131]:
# map with trace instance chained to ord function
class Trace:    
    def __init__(self):                                                     
        self.enabled = True                                                 
    def __call__(self, f):                                                  
        def wrap(*args, **kwargs):                                          
            if self.enabled:                                                
                print('Calling {}'.format(f))                               
            return f(*args, **kwargs)                                       
        return wrap   

In [132]:
result = map(Trace()(ord), 'The quick brown fox') 

In [133]:
result

In [134]:
next(result)

Calling <built-in function ord>


84

In [135]:
# example code of multiple arguments in mapped function
# with multiple input sequences

sizes = ['small', 'medium', 'large'] 
colors = ['lavender', 'teal', 'burnt orange']                               
animals = ['koala', 'platypus', 'salamander']                               
def combine(size, color, animal):                                           
    return '{} {} {}'.format(size, color, animal)                           
                                                                            
list(map(combine, sizes, colors, animals))                                  

['small lavender koala',
 'medium teal platypus',
 'large burnt orange salamander']

In [136]:
# using itertools.count() to automatically generate count values
# based on the size of other input sequences
def combine(quantity, size, color, animal):                                 
    return '{} x {} {} {}'.format(quantity, size, color, animal)            
                                                                            
import itertools                                                            
list(map(combine, itertools.count(), sizes, colors, animals))               

['0 x small lavender koala',
 '1 x medium teal platypus',
 '2 x large burnt orange salamander']

#####  filter()
  + removes elements from a sequence which don't meet some criteria
  + applies a predicate function to each element
  + produces its results lazily
  + only take a single input sequence and the predicate function only take a single argument
  * if you put None as the first argument without providing a specific predicate function, filter will just output all elments that themselves evaluated as True. Zero, empty string, False and empty list will be filtered out

##### filter() examples

In [137]:
positives =filter(lambda x: x > 0, [1, -5, 0, 6, -2, 8])
positives

In [138]:
list(positives)

[1, 6, 8]

In [140]:
trues = filter(None, [0, 1, False, True, [], [1, 2, 3], '', 'hello'])
list(trues)

[1, True, [1, 2, 3], 'hello']

##### functools.reduce()
* repeatedly applies a two-argument function to an accumulated value and the next element from a sequence
* the function is called multiple times until the sequence is reduced to a single value
* the initial value can be the first element in the input sequence or an optional argument
* the final accumulated or reduced value is return as a single value
* if you pass an empty sequence, it will raise a TypeError
* if you pass a one element sequence, it will return the element without calling the reducing funtion
* it accepts an optional initial value, if the input sequence is empty, the initial value will be returned
  + use 0 and 1 as inital values for addition and multiplication, respectively

##### functools.reduce examples

In [142]:
from functools import reduce
import operator                                                             
reduce(operator.add, [1, 2, 3, 4, 5])          

15

In [143]:
numbers = [1, 2, 3, 4, 5]                                                   
accumulator = operator.add(numbers[0], numbers[1])                          
for item in numbers[2:]:                                                    
    accumulator = operator.add(accumulator, item)                           
                                                                            
accumulator 

15

In [144]:
# demonstrate the process of reduce
# the fucntion is called multiple time 
# until the sequence is reduced to a single value
def mul(x, y):                                                              
    print('mul {} {}'.format(x, y))                                         
    return x * y                                                            
                                                                            
reduce(mul, range(1, 10))                     

mul 1 2
mul 2 3
mul 6 4
mul 24 5
mul 120 6
mul 720 7
mul 5040 8
mul 40320 9


362880

In [146]:
# inital value of 0
values = [1, 2, 3]                                                          
reduce(operator.add, values, 0)                                             

6

In [147]:
# initial value with an empty sequence
values = []                                                                 
reduce(operator.add, values, 0)                                             

0

In [148]:
# use 0 as the initial value for addition                                                                             
values = [1, 2, 3]                                                          
reduce(operator.add, values, 0)                                             

6

In [149]:
# use 1 as the initial value for multiplication               
values = [1, 2, 3]                                                          
reduce(operator.mul, values, 1)    

6

##### Map-reduce
* in the following example, we define a function, count_words to count frequency of words in a sentence
* we then apply the function to a list of sentences by map function
* so we now have the frequencies of words in each sentence, we then use reduce to sum the frequencies for each word

##### Map-reduce examples

In [150]:
# convert the input string into space separated words
# and count the frequence, save the results in a dictionary
def count_words(doc):                                                       
    normalised_doc = ''.join(c.lower() if c.isalpha() else ' ' for c in doc)
    frequencies = {}                                                        
    for word in normalised_doc.split():                                     
        frequencies[word] = frequencies.get(word, 0) + 1                    
    return frequencies                                                      
                                                                            
count_words('It was the best of times, it was the worst of times.')         

{'it': 2, 'was': 2, 'the': 2, 'best': 1, 'of': 2, 'times': 2, 'worst': 1}

In [152]:
# given a document list, apply count_words to each element    
documents = [                                                               
    'It was the best of times, it was the worst of times.',
    'I went to the woods because I wished to live deliberately, to front only the essential facts of life...',                                              
    'Friends, Romans, countrymen, lend me your ears; I come to bury Caesar, not to praise him.',                                                            
    'I do not like green eggs and ham. I do not like them, Sam-I-Am.',      
]

counts = map(count_words, documents)

In [153]:
# given two frequency dictionary, combine them to one dictionary
def combine_counts(d1, d2):                                                 
    d = d1.copy()                                                           
    for word, count in d2.items():                                          
        d[word] = d.get(word, 0) + count                                    
    return d                                                                
                                                                            
from functools import reduce                                                
total_counts = reduce(combine_counts, counts)                               
total_counts                                        

{'it': 2,
 'was': 2,
 'the': 4,
 'best': 1,
 'of': 3,
 'times': 2,
 'worst': 1,
 'i': 6,
 'went': 1,
 'to': 5,
 'woods': 1,
 'because': 1,
 'wished': 1,
 'live': 1,
 'deliberately': 1,
 'front': 1,
 'only': 1,
 'essential': 1,
 'facts': 1,
 'life': 1,
 'friends': 1,
 'romans': 1,
 'countrymen': 1,
 'lend': 1,
 'me': 1,
 'your': 1,
 'ears': 1,
 'come': 1,
 'bury': 1,
 'caesar': 1,
 'not': 3,
 'praise': 1,
 'him': 1,
 'do': 2,
 'like': 2,
 'green': 1,
 'eggs': 1,
 'and': 1,
 'ham': 1,
 'them': 1,
 'sam': 1,
 'am': 1}

### Multi-input and nested comprehension
* comprehensions can have multiple input iterables and if-clauses

#### Comprehensions

In [159]:
# list comprehension
l = [i * 2 for i in range(10)]
l

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [156]:
# dictionary comprehension
d = {i: i * 2 for i in range(10)}                                           
type(d)                                                                     

dict

In [157]:
 # set comprehension                                                           
s = {i for i in range(10)}                                                  
type(s)                                                                     

set

In [158]:
# generator comprehension                                                              
g = (i for i in range(10))                                                  
type(g)

generator

#### multi-input and multi-if comprehensions
* the way to read this is as as set of nested for loops where the later for clauses are nested inside the earlier for clauses, and the result expression of the comprehension is executed inside the inner most, or last for loop 
* since they are treated as nested for loops, the later clauses can refer variables in the earlier clauses

##### Example of multi-input

In [160]:
[(x, y) for x in range(5) for y in range(5)]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [171]:
# the equivalent implementation of the above 
# multi-input list comprehension
points = []                                                                 
for x in range(5):                                                          
    for y in range(5):                                                      
        points.append((x, y))  

##### Example of multi-input with multiple if

In [164]:
# multi-input with multi-if comprehension 
# is the same as nested loop 
values = [x / (x - y)                                                       
          for x in range(100)                                               
          if x > 50                                                         
          for y in range(100)                                               
          if x - y != 0]

In [170]:
# equvalent to the above list comprehension
values = []                                                                 
for x in range(100):                                                        
    if x > 50:                                                              
        for y in range(100):                                                
            if x - y != 0:                                                  
                values.append(x / (x - y))                         
                                                                            

##### Example of later clause refer variables in earlier clause

In [174]:
result = [(x, y) for x in range(10) for y in range(x)]  
result

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (9, 0),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8)]

In [168]:
# equvalent to the above list comprehension
result = []                                                                 
for x in range(10):                                                         
    for y in range(x):                                                      
        result.append((x, y))     

#### Nested Comprehensions
* we can use comprehensions in output expressions
* same to multi-input comprehension that involves multiple for loops, the structures they produce are different

In [173]:
vals = [[y * 3 for y in range(x)] for x in range(10)]                       
vals

[[],
 [0],
 [0, 3],
 [0, 3, 6],
 [0, 3, 6, 9],
 [0, 3, 6, 9, 12],
 [0, 3, 6, 9, 12, 15],
 [0, 3, 6, 9, 12, 15, 18],
 [0, 3, 6, 9, 12, 15, 18, 21],
 [0, 3, 6, 9, 12, 15, 18, 21, 24]]

In [175]:
outer = [] 
for x in range(10):                                                         
    inner = []                                                              
    for y in range(x):                                                      
        inner.append(y * 3)                                                 
    outer.append(inner)

[[],
 [0],
 [0, 3],
 [0, 3, 6],
 [0, 3, 6, 9],
 [0, 3, 6, 9, 12],
 [0, 3, 6, 9, 12, 15],
 [0, 3, 6, 9, 12, 15, 18],
 [0, 3, 6, 9, 12, 15, 18, 21],
 [0, 3, 6, 9, 12, 15, 18, 21, 24]]

#### Other conmprehensions
* the rules for list comprehensions also apply to other comprehensions

In [176]:
{x * y for x in range(10) for y in range(x)}

{0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 12,
 14,
 15,
 16,
 18,
 20,
 21,
 24,
 27,
 28,
 30,
 32,
 35,
 36,
 40,
 42,
 45,
 48,
 54,
 56,
 63,
 72}

In [177]:
g = ((x, y) for x in range(10) for y in range(x))